In [ ]:
from pathlib import Path

import yaml
from deepdiff import DeepDiff

In [ ]:
paper_plots_path = Path("/Users/jorisg/Desktop/upper_limb/paper_figures_revision")

intact = [
    "P_149",
    "P_238",
    "P_407",
    "P_426",
    "P_577",
    "P_668",
    "P_711",
    "P_950",
]
amputees = ["P7_453", "P6_820"]
participants = intact + amputees
targets = [
    "indexAng",
    "midAng",
    "ringAng",
    "pinkyAng",
    "thumbInPlaneAng",
    "thumbOutPlaneAng",
    "wristFlex",
]
recordings = [
    "thumbFlEx",
    "thumbAbAd",
    "indexFlEx",
    "mrpFlEx",
    "fingersFlEx",
    "wristFlEx",
    "handOpCl",
    "pinchOpCl",
    "pointOpCl",
]
test_recordings = ["keyOpCl", "wristFlHandCl", "indexFlDigitsEx"]

channel_names = [
    "FPL",
    "EPL",
    "FDSr",
    "EIP",
    "FDSh",
    "EDC",
    "FCU",
    "ECU",
]

In [ ]:
base_path = Path("data/P_149/configs/modular.yaml")
with open(base_path, "r") as f:
    base_config = yaml.safe_load(f)

all_configs = [Path(f"data/{p}/configs/modular.yaml") for p in participants]
all_configs = [Path(p) for p in all_configs if Path(p) != base_path]

for config_path in all_configs:
    with open(config_path, "r") as f:
        other_config = yaml.safe_load(f)
    diff = DeepDiff(base_config, other_config, ignore_order=True)
    print(f"\n--- Differences with {config_path} ---")
    if diff:
        print(diff)
    else:
        print("No differences.")

-> P7_453 is Right, all others are Left


In [ ]:
def copy_initialization_config(participants, base_path):
    base_config_path = Path(base_path)
    with open(base_config_path, "r") as f:
        base_config = yaml.safe_load(f)

    for participant in participants:
        participant_config_path = Path(
            f"data/{participant}/configs/modular_initialization.yaml"
        )
        if not participant_config_path.exists():
            # Create the directory if it doesn't exist
            # participant_config_path.parent.mkdir(parents=True, exist_ok=True)
            # Modify the base config for the current participant
            base_config["name"] = participant
            with open(participant_config_path, "w") as f:
                yaml.dump(base_config, f)
            print(f"Copied config to {participant_config_path}")


copy_initialization_config(
    participants, "data/<person_ID>/configs/modular_initialization.yaml"
)
# TODO: change the targets for P7_453 to right

In [ ]:
import subprocess

for participant in participants:
    hand = "Right" if participant == "P7_453" else "Left"
    print(f"Running training for {participant} with intact hand {hand}")

    # Run the command using subprocess
    subprocess.run(
        [
            "python",
            "s4_train.py",
            "--person_dir",
            participant,
            "--intact_hand",
            hand,
            "--config_name",
            "modular_initialization",
            "-hs",
        ]
    )

In [ ]:
import pandas as pd

import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("MIT_biomech/initialization_experiments")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

# runs_df.to_csv("project.csv")

In [ ]:
runs_df["total_val_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_val_loss", None)
)
# total_test_loss
runs_df["total_test_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_test_loss", None)
)
runs_df["init"] = runs_df["config"].apply(
    lambda x: x.get("activation_model").get("init", None)
)

In [ ]:
runs_df.drop(columns=["summary", "config"], inplace=True)
runs_df.rename(columns={"name": "participant"}, inplace=True)

In [ ]:
runs_df["total_combined_loss"] = (
    runs_df["total_val_loss"] * 9 + runs_df["total_test_loss"] * 3
) / 12

In [ ]:
# group by participant and init, then get the mean and std of total_val_loss
grouped_df = (
    runs_df.groupby(["participant", "init"])
    .agg(
        mean_val_loss=("total_val_loss", "mean"),
        std_val_loss=("total_val_loss", "std"),
        mean_test_loss=("total_test_loss", "mean"),
        std_test_loss=("total_test_loss", "std"),
        mean_combined_loss=("total_combined_loss", "mean"),
        std_combined_loss=("total_combined_loss", "std"),
    )
    .reset_index()
)

In [ ]:
grouped_df_amputee = grouped_df[grouped_df["participant"].isin(amputees)].copy()
grouped_df_intact = grouped_df[grouped_df["participant"].isin(intact)].copy()

In [ ]:
# average over all participants
average_df_amputee = (
    grouped_df_amputee.groupby("init")
    .agg(
        mean_mean_val_loss=("mean_val_loss", "mean"),
        mean_std_val_loss=("std_val_loss", "mean"),
        mean_mean_test_loss=("mean_test_loss", "mean"),
        mean_std_test_loss=("std_test_loss", "mean"),
        mean_mean_combined_loss=("mean_combined_loss", "mean"),
        mean_std_combined_loss=("std_combined_loss", "mean"),
    )
    .reset_index()
)
average_df_intact = (
    grouped_df_intact.groupby("init")
    .agg(
        mean_mean_val_loss=("mean_val_loss", "mean"),
        mean_std_val_loss=("std_val_loss", "mean"),
        mean_mean_test_loss=("mean_test_loss", "mean"),
        mean_std_test_loss=("std_test_loss", "mean"),
        mean_mean_combined_loss=("mean_combined_loss", "mean"),
        mean_std_combined_loss=("std_combined_loss", "mean"),
    )
    .reset_index()
)

In [ ]:
average_df_intact.index = average_df_intact["init"]
average_df_amputee.index = average_df_amputee["init"]
average_df_intact.drop(columns=["init"], inplace=True)
average_df_amputee.drop(columns=["init"], inplace=True)

In [ ]:
# round to 3 decimal places
average_df_amputee = average_df_amputee.round(3)
average_df_intact = average_df_intact.round(3)

In [ ]:
# transpose the dataframes
average_df_amputee = average_df_amputee.T
average_df_intact = average_df_intact.T

In [ ]:
# Move the distinction between mean and std to the column names
# The new columns will be MultiIndex: (Initialization_method, Statistic)
new_df = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [["Combined", "New Movements", "Known Movements"], ["Intact", "Amputation"]],
        names=["Data Set", "Cohort"],
    ),
    columns=pd.MultiIndex.from_product(
        [average_df_amputee.columns, ["mean", "std"]],
        names=["Initialization_method", "Statistic"],
    ),
)
new_df

In [ ]:
for column in average_df_amputee.columns:
    new_df.loc[("Combined", "Amputation"), (column, "mean")] = average_df_amputee.loc[
        "mean_mean_combined_loss", column
    ]
    new_df.loc[("Combined", "Amputation"), (column, "std")] = average_df_amputee.loc[
        "mean_std_combined_loss", column
    ]
    new_df.loc[("New Movements", "Amputation"), (column, "mean")] = (
        average_df_amputee.loc["mean_mean_test_loss", column]
    )
    new_df.loc[("New Movements", "Amputation"), (column, "std")] = (
        average_df_amputee.loc["mean_std_test_loss", column]
    )
    new_df.loc[("Known Movements", "Amputation"), (column, "mean")] = (
        average_df_amputee.loc["mean_mean_val_loss", column]
    )
    new_df.loc[("Known Movements", "Amputation"), (column, "std")] = (
        average_df_amputee.loc["mean_std_val_loss", column]
    )
for column in average_df_intact.columns:
    new_df.loc[("Combined", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_combined_loss", column
    ]
    new_df.loc[("Combined", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_combined_loss", column
    ]
    new_df.loc[("New Movements", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_test_loss", column
    ]
    new_df.loc[("New Movements", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_test_loss", column
    ]
    new_df.loc[("Known Movements", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_val_loss", column
    ]
    new_df.loc[("Known Movements", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_val_loss", column
    ]

In [ ]:
# reorder Initialization_method: [default_all, default_0, default, kaiming_normal, xavier_normal, orthogonal]
new_df = new_df.reindex(
    columns=[
        ("default_all", "mean"),
        ("default_all", "std"),
        ("default_0", "mean"),
        ("default_0", "std"),
        ("default", "mean"),
        ("default", "std"),
        ("kaiming_normal", "mean"),
        ("kaiming_normal", "std"),
        ("xavier_normal", "mean"),
        ("xavier_normal", "std"),
        ("orthogonal", "mean"),
        ("orthogonal", "std"),
    ]
)

In [ ]:
new_df.to_csv(paper_plots_path / "initialization_experiment_results.csv")

In [ ]:
# replace this: [default_all, default_0, default, kaiming_normal, xavier_normal, orthogonal]
# with a multi_index:
# LeakyReLu Initialization: [Default, Default, Default, Kaiming Normal, Xavier Normal, Orthogonal]
# Sigmoid Initialization: [Default, Default, Xavier Normal, Xavier Normal, Xavier Normal, Xavier Normal]
# Bias Initialization: [Default, Zero, Zero, Zero, Zero, Zero]


In [ ]:
new_df